## An Automated Portfolio Trading System with Feature Preprocessing and Recurrent Reinforcement Learning

### Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.data import Data, get_buy_and_hold_df, INDICATORS

from lib.model import RRL, train, validation

from lib.backtest import (
    run_rrl_strategy, 
    create_backtest_dataset, 
    generate_layout, 
    generate_nrows, 
    generate_subplot_titles, 
    init_subplots, 
    populate_subplots,
    plot_cumulative_profits, 
    generate_df_barplot, 
    make_cumrets_barplot,
    plot_avg_portfolio_allocation, 
)

from lib.metrics import (
    calc_cumulative_profits, 
    calc_sharpe_ratio, 
)

from lib import save_to_yaml, read_yaml

In [3]:
import numpy as np 
import pandas as pd

from pickle import load, dump

### Data

#### Settings

**Assets**

|  Ticker | Company  |
|---|---|
| XOM | Exxon Mobil Corporation |
| VZ | Verizon Communications Inc. |
| NKE | Nike, Inc. |
| AMAT | Applied Materials, Inc. |
| MCD | McDonald's Corporation |
| MSFT | Microsoft Corporation |
| AAP | Advance Auto Parts, Inc. |
| NOV | Nov, Inc. |

In [4]:
# assets in the portfolio
ASSETS = ["XOM", "VZ", "NKE", "AMAT", "MCD", "MSFT", "AAP", "NOV"]
n_assets = 8
# assets = np.random.choice(ASSETS, size=n_assets, replace=False).tolist()
assets = ["AMAT", "MSFT", "XOM", "NOV"]

# date settings
start_date = "2009-12-31"
end_date = "2017-12-29"
window_size = 100

# technical indicators
momentum = [
    "MOM",
    "MACD",
    "MFI",
    "RSI"
]

volatility = [
    "ATR",
    "NATR"
]

cycle = [ 
    # "HTDCP",
    # "HTS",
    # "HTTMMM"      
]
# nb: cycle indicators seem to decrease the performance of the trading system

volume = [
    "CO",
    "OBV"
]

indicators = momentum + volatility + cycle + volume
indicators

# number of principal components (None means no PCA)
pca_ncp = 5

# boolean indicating whether to apply discrete wavelet transformation
dwt = True

#### Instantiate `Data` object

In [5]:
data = Data(start_date, end_date, window_size, assets, indicators, pca_ncp, dwt)
data

[*********************100%***********************]  4 of 4 completed


Data(start_date='2009-12-31', end_date='2017-12-29', window_size=100, assets=['AMAT', 'MSFT', 'XOM', 'NOV'], indicators=['MOM', 'MACD', 'MFI', 'RSI', 'ATR', 'NATR', 'CO', 'OBV'], pca_ncp=5, discrete_wavelet=True)

In [9]:
# save data object into yaml configuration file 

# save_to_yaml(obj=data, path="./config/data.yaml")

### `RRL` - Training and validation

In [6]:
initial_invest = 100
n_epochs = 100

#### No transaction fees 

$\delta=0$

In [8]:
n_features = data.pca_ncp
# n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features) 
print(rrl) 

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

file_path = "./backup/rrl_no_fees_pca_dwt_1.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=4, n_features=5, delta=0, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  16%|█▌        | 16/100 [00:00<00:02, 40.49it/s, sharpe_ratio=-.0484]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 1.97484393694706
Cumulative profits: 111.66503196617168
Training window: 2010-06-30/2010-10-14


Training in progress...:  26%|██▌       | 26/100 [00:00<00:02, 33.69it/s, sharpe_ratio=0.0564]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 2.467531239032628
Cumulative profits: 124.07385036688456
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: -0.45443021861012434
Cumulative profits: 233.93301077901512
Training window: 2017-03-31/2017-07-18


Training in progress...:  43%|████▎     | 43/100 [00:01<00:01, 41.70it/s, sharpe_ratio=-.0772]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 1.2873098719690004
Cumulative profits: 248.36723079849776
Training window: 2017-08-23/2017-12-07


Training in progress...:   4%|▍         | 4/100 [00:00<00:03, 28.87it/s, sharpe_ratio=0.0669]


Less than 30 observations in the validation set after preprocessing.


#### Transactions fees 

$\delta \in \{0.001, 0.003, 0.005\}$

In [9]:
n_features = data.pca_ncp
# n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features, delta=.001) 
print(rrl)

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

file_path = "./backup/rrl_10bps_pca_dwt_1.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=4, n_features=5, delta=0.001, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:   7%|▋         | 7/100 [00:00<00:02, 39.02it/s, sharpe_ratio=-.0638]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 3.0502605467889112
Cumulative profits: 120.0552594850005
Training window: 2010-06-30/2010-10-14


Training in progress...:  21%|██        | 21/100 [00:00<00:02, 35.36it/s, sharpe_ratio=0.184]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 2.7245456279638094
Cumulative profits: 137.79151716340257
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: 1.3222904007437857
Cumulative profits: 260.3160910316801
Training window: 2017-03-31/2017-07-18


Training in progress...:   1%|          | 1/100 [00:00<00:04, 23.50it/s, sharpe_ratio=0.0723]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 3.8928362917635324
Cumulative profits: 302.0903720134102
Training window: 2017-08-23/2017-12-07


Training in progress...:  21%|██        | 21/100 [00:00<00:02, 38.65it/s, sharpe_ratio=0.233]


Less than 30 observations in the validation set after preprocessing.


In [10]:
n_features = data.pca_ncp
# n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features, delta=.003) 
print(rrl)

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

file_path = "./backup/rrl_30bps_pca_dwt_1.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=4, n_features=5, delta=0.003, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  11%|█         | 11/100 [00:00<00:02, 30.36it/s, sharpe_ratio=-.0519]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 2.29304885947267
Cumulative profits: 114.10591418916447
Training window: 2010-06-30/2010-10-14


Training in progress...:  72%|███████▏  | 72/100 [00:01<00:00, 38.63it/s, sharpe_ratio=0.166]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 1.8453380100099976
Cumulative profits: 123.47499837874398
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: -0.20010369374514012
Cumulative profits: 178.7569194913119
Training window: 2017-03-31/2017-07-18


Training in progress...:   4%|▍         | 4/100 [00:00<00:02, 43.51it/s, sharpe_ratio=-.00495]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 2.656044458695658
Cumulative profits: 197.94934449191018
Training window: 2017-08-23/2017-12-07


Training in progress...:   5%|▌         | 5/100 [00:00<00:02, 41.62it/s, sharpe_ratio=0.161]

Less than 30 observations in the validation set after preprocessing.


In [11]:
n_features = data.pca_ncp
# n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features, delta=.005) 
print(rrl)

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

file_path = "./backup/rrl_50bps_pca_dwt_1.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=4, n_features=5, delta=0.005, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  14%|█▍        | 14/100 [00:00<00:02, 40.50it/s, sharpe_ratio=-.0578]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 1.8286766459937251
Cumulative profits: 110.73451466312434
Training window: 2010-06-30/2010-10-14


Training in progress...:  21%|██        | 21/100 [00:00<00:02, 33.09it/s, sharpe_ratio=0.17]  


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 3.845880676006762
Cumulative profits: 131.54015931306157
Training window: 2010-11-19/2011-03-09


...

Training in progress...:   1%|          | 1/100 [00:00<00:04, 24.15it/s, sharpe_ratio=0.0521]

Validation window: 2017-08-23/2017-12-07


Sharpe ratio on validation set: 3.049538423836497
Cumulative profits: 250.37597871485679
Training window: 2017-08-23/2017-12-07


Training in progress...:   1%|          | 1/100 [00:00<00:03, 26.97it/s, sharpe_ratio=0.208]

Less than 30 observations in the validation set after preprocessing.


### Backtest

In [12]:
version = "pca_dwt_1"
file_names = [f"rrl_{fees}_{version}" for fees in ("no_fees", "10bps", "30bps", "50bps")]

path = "./backup/"
results = {
    f.replace("results_", ""): load(open(f"{path}{f}.pkl", "rb"))
    for f in file_names
}

In [13]:
dfs_backtest = {k: create_backtest_dataset(v) for k, v in results.items()}

#### Cumulative profits for different settings

In [14]:
initial_invest = 100

In [15]:
buy_and_hold = get_buy_and_hold_df(data.df)
buy_and_hold_sr = calc_sharpe_ratio(returns=buy_and_hold.returns, window_size=252)

In [16]:
layout = generate_layout(
    fig_height=500, 
    fig_width=1100, 
    fig_title="", 
    x_axis_args={"gridcolor": "lightgrey"}, 
    y_axis_args={"gridcolor": "lightgrey", "title": {"text": "$"}}
)
fig = plot_cumulative_profits(
    dfs_backtest, 
    buy_and_hold, 
    buy_and_hold_sr, 
    initial_invest, 
    layout)

fig.show()

In [22]:
# fig.write_image("imgs/cum-profits-rrl-pca-dwt-3.png")
# fig.write_html("imgs/cum-profits-rrl-pca-dwt-3.html", full_html=False, include_plotlyjs='cdn')

#### Analysis per trading window for one setting

In [17]:
setting = "rrl_no_fees_pca_dwt_1"

df = dfs_backtest[setting]
batch_names = np.unique(df["Batch"])

In [18]:
metric ="Cumulative returns"
fig_title = f"{metric} per trading window | {setting}"

ncols = 4
nrows = generate_nrows(len(batch_names), ncols)

titles = generate_subplot_titles(batch_names)

fig = init_subplots(nrows, ncols, titles)
fig = populate_subplots(fig, df, batch_names, titles, ncols, y_varname=metric, initial_invest=initial_invest)

layout = generate_layout(
    fig_height=1200, 
    fig_width=1400, 
    fig_title=fig_title)
    
fig.update_layout(layout)

In [19]:
df_barplot = generate_df_barplot(df)
title = f"% Cumulative returns at the end of each trading window | {setting}"

layout = generate_layout(
    fig_height=500, 
    fig_width=1100, 
    fig_title=title,
    x_axis_args={"title": {"text": "Trading window"}}, 
    y_axis_args={"gridcolor": "lightgrey", "title": {"text": "%"}}
)
fig = make_cumrets_barplot(df_barplot, layout)

fig.show()

#### Portfolio allocation

In [20]:
layout = generate_layout(
    fig_height=500, 
    fig_width=700, 
    fig_title="Average portfolio allocation over trading windows"
)

fig = plot_avg_portfolio_allocation(results[setting], layout)

fig.show()